# 01. Importing libraries

In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

# 02. Importing data

In [2]:
path = r'/Users/marcela/Achievement 4'

In [3]:
ords_prods = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_all.pkl'))

## Set PD code

In [4]:
# Creating PD setting, display max columns
pd.set_option('display.max_columns', 1000)

# 03. Security implications

In [5]:
# Reviewing column headers and first 5 rows for security implications
ords_prods.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,_merge,product_name,aisle_id,department_id,prices,price_range_loc,busiest_day,busiest_period_of_day,max_order,loyalty_flag,spending,spending_flag,order_frequency,order_frequency_flag,first_name,surname,gender,state,age,date_joined,num_of_dependants,marital_status,income
0,2539329,1,1,2,8,NaN,196,1,0,both,Soda,77,7,9.0,Mid-range product,Regularly busy,Average orders,10,New customer,6.367797,low spender,20.5,Non-frequent customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
1,2398795,1,2,3,7,15.0,196,1,1,both,Soda,77,7,9.0,Mid-range product,Least busy,Average orders,10,New customer,6.367797,low spender,20.5,Non-frequent customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
2,473747,1,3,3,12,21.0,196,1,1,both,Soda,77,7,9.0,Mid-range product,Least busy,Average orders,10,New customer,6.367797,low spender,20.5,Non-frequent customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
3,2254736,1,4,4,7,29.0,196,1,1,both,Soda,77,7,9.0,Mid-range product,Least busy,Average orders,10,New customer,6.367797,low spender,20.5,Non-frequent customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
4,431534,1,5,4,15,28.0,196,1,1,both,Soda,77,7,9.0,Mid-range product,Least busy,Average orders,10,New customer,6.367797,low spender,20.5,Non-frequent customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423


# 04. Analysis of Regional Sales

## Creating a Regional Flag

In [6]:
# Creating LOC for the Northeast region
ords_prods.loc[ords_prods['state'].isin(['Maine', 'New Hampshire', 'Vermont', 'Massachusetts', 'Rhode Island', 'Connecticut', 'New York', 'Pennsylvania', 'New Jersey']), 'regional_flag'] = 'Northeast'

/var/folders/gz/793390fs5lz_gt8d667yd0qw0000gn/T/ipykernel_2334/2378537938.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'Northeast' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  ords_prods.loc[ords_prods['state'].isin(['Maine', 'New Hampshire', 'Vermont', 'Massachusetts', 'Rhode Island', 'Connecticut', 'New York', 'Pennsylvania', 'New Jersey']), 'regional_flag'] = 'Northeast'


In [7]:
# Creating LOC for the Midwest region
ords_prods.loc[ords_prods['state'].isin(['Wisconsin', 'Michigan', 'Illinois', 'Indiana', 'Ohio', 'North Dakota', 'South Dakota', 'Nebraska', 'Kansas', 'Minnesota', 'Iowa', 'Missouri']), 'regional_flag'] = 'Midwest'

In [8]:
# Creating LOC for the South region
ords_prods.loc[ords_prods['state'].isin(['Delaware', 'Maryland', 'District of Columbia', 'Virginia', 'West Virginia', 'North Carolina', 'South Carolina', 'Georgia', 'Florida', 'Kentucky', 'Tennessee', 'Mississippi', 'Alabama', 'Oklahoma', 'Texas', 'Arkansas', 'Louisiana']), 'regional_flag'] = 'South'

In [9]:
# Creating LOC for the West region
ords_prods.loc[ords_prods['state'].isin(['Idaho', 'Montana', 'Wyoming', 'Nevada', 'Utah', 'Colorado', 'Arizona', 'New Mexico', 'Alaska', 'Washington', 'Oregon', 'California', 'Hawaii']), 'regional_flag'] = 'West'

In [10]:
# Checking regional Flag value counts
ords_prods['regional_flag'].value_counts(dropna=False)

regional_flag
South        10801610
West          8300445
Midwest       7603810
Northeast     5728347
Name: count, dtype: int64

In [11]:
# Total Row count for Regional_flag
ords_prods['regional_flag'].count()

32434212

In [12]:
ords_prods['regional_flag'] = ords_prods['regional_flag'].astype(str)
ords_prods.loc[ords_prods['state'].isin(['Maine', 'New Hampshire', 'Vermont', 'Massachusetts', 'Rhode Island', 'Connecticut', 'New York', 'Pennsylvania', 'New Jersey']), 'regional_flag'] = 'Northeast'

## Determining whether there’s a difference in spending habits between the different U.S. regions

In [13]:
# Creating a crosstab 
region_crosstab=pd.crosstab(ords_prods['regional_flag'],ords_prods['spending_flag'],dropna=False)

In [14]:
# Checking
region_crosstab

spending_flag,high spender,low spender
regional_flag,,
Midwest,156129,7447681
Northeast,108343,5620004
South,210182,10591428
West,160807,8139638


# 05. Creating a exclusion flag for low-activity customers

## Creating a exclusionary flag for low-spenders

In [15]:
# Creating max column for client orders
ords_prods['max_order'] = ords_prods.groupby(['user_id'])['order_number'].transform('max')

In [16]:
# Creating a exclusionary row for low activity and active customers
ords_prods['Customer_activity'] = np.where(ords_prods['max_order'] < 5, 'Low_activity', 'Active')

In [17]:
ords_prods['Customer_activity'].value_counts()

Customer_activity
Active          30992664
Low_activity     1441548
Name: count, dtype: int64

In [18]:
# Creating subset of low_activity customers
orders_products_Low_activity = ords_prods[ords_prods['Customer_activity'] == 'Low_activity']

In [19]:
# Creating subset of active customers
orders_products_active = ords_prods[ords_prods['Customer_activity'] == 'Active']

In [20]:
# Checking
orders_products_Low_activity.shape

(1441548, 34)

In [21]:
# Checking
orders_products_active.shape

(30992664, 34)

In [24]:
# Matching total data set row count
1441548 + 30992664

32434212

## Exporting Active Members data set as orders_products_active.pkl

In [25]:
orders_products_active.to_pickle(os.path.join(path,'02 Data', 'Prepared Data', 'orders_products_active.pkl'))